<a href="https://colab.research.google.com/github/nirvanesque/interesting-notebooks/blob/main/MedicalChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [ ]:
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Setup Huggingface token**

In [ ]:
from google.colab import userdata

In [ ]:
# Or use `os.getenv('HUGGINGFACEHUB_API_TOKEN')` to fetch an environment variable.
import os
from getpass import getpass

HF_API_KEY = userdata.get("HF_API_KEY")
os.environ["HF_API_KEY"] = "HF_API_KEY"

# **Import Document**

In [ ]:
loader = PyPDFDirectoryLoader('/content/drive/My Drive/Data')
docs = loader.load()

In [ ]:
docs

[Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 0}, page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute\n'),
 Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 1}, page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. D EPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood InstituteNIH Publication No. 06-5269December 2005'),
 Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 2}, page_content='U.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of HealthNational Heart, Lung, and Blood InstituteWritten by: Marian Sandmaier'),
 Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 3}, page_content='Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . .

# **Text Splitting - Chunking**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks)

747

In [ ]:
chunks[0]

Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 0}, page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute')

In [ ]:
chunks[1]

Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 1}, page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. D EPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood InstituteNIH Publication No. 06-5269December 2005')

In [ ]:
chunks[2]

Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 2}, page_content='U.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of HealthNational Heart, Lung, and Blood InstituteWritten by: Marian Sandmaier')

In [ ]:
chunks[3]

Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 3}, page_content='Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nWhat You Need To Know About Heart Disease . . . . . . . . . . . . . . . . . . . . . . 3')

In [ ]:
chunks[4]

Document(metadata={'source': '/content/drive/My Drive/Data/healthyheart.pdf', 'page': 3}, page_content='What Is Heart Disease?. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3Who Is at Risk? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4How Risk Works . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .')

# **Embeddings**

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

<ipython-input-16-e470369191f5>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Vector store**

In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
vectorstore

In [ ]:
query = "who is at risk of heart disease"
search = vectorstore.similarity_search(query)

In [ ]:
to_markdown(search[0].page_content)

> an important risk factor for heart disease and other medicaldisorders.

# **Retriever**

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

In [ ]:
retriever.get_relevant_documents(query)

<ipython-input-22-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 13, 'source': '/content/drive/My Drive/Data/healthyheart.pdf'}, page_content='an important risk factor for heart disease and other medicaldisorders.'),
 Document(metadata={'page': 49, 'source': '/content/drive/My Drive/Data/healthyheart.pdf'}, page_content='Some people may need to take additional steps to prevent heart'),
 Document(metadata={'page': 15, 'source': '/content/drive/My Drive/Data/healthyheart.pdf'}, page_content='important risk factorsfor heart disease and how each of them affects your health.'),
 Document(metadata={'page': 24, 'source': '/content/drive/My Drive/Data/healthyheart.pdf'}, page_content='high risk for a heart attack—at the same level of risk as someone who has heart disease.Also, even though overweight and physical inactivity are not on this list of risk factors,they are conditions that raise your risk for heart disease and need to be corrected.'),
 Document(metadata={'page': 8, 'source': '/content/drive/My Drive/Data/healthyheart.p

# **Large Language Model - Open Source**

In [ ]:
llm = LlamaCpp(
    model_path= "/content/drive/MyDrive/Model/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.3,
    max_tokens=2048,
    top_p=1)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/Model/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attentio

# **RAG Chain**

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
template = """
<|context|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {'context': retriever, 'query': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = rag_chain.invoke("Why should I care about my heart health?")


llama_print_timings:        load time =    4600.93 ms
llama_print_timings:      sample time =      38.67 ms /    56 runs   (    0.69 ms per token,  1448.19 tokens per second)
llama_print_timings: prompt eval time =   26918.96 ms /    61 tokens (  441.29 ms per token,     2.27 tokens per second)
llama_print_timings:        eval time =   41339.95 ms /    55 runs   (  751.64 ms per token,     1.33 tokens per second)
llama_print_timings:       total time =   68360.86 ms /   116 tokens


In [ ]:
to_markdown(response)

> Because your heart is a vital organ in your body, it pumps blood to all of your organs. If you take good care of your heart by eating healthy food, exercising regularly and not smoking, you can prevent heart disease and live a long and healthy life.